In [ ]:
from pyhive.hive import connect
import os
import pandas as pd
import numpy as np
import joblib
from sklearn import metrics
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn import preprocessing
from sklearn.ensemble import IsolationForest
import random

In [ ]:
# 将数据库中的表格导入开发环境中
keytab_file='***@cnpc.com.cn.keytab'
user='gaohaojie'
host='**.**.***.***'
port=10000
active_str='kinit -kt {0} {1}'.format(keytab_file,user)
os.system(active_str)
con=connect(host=host,port=port,database='***',auth='KERBEROS',kerberos_service_name="hive")
cursor=con.cursor()
dim_sql = "select * from ***"
cursor.execute(dim_sql)
curinfo = cursor.fetchall()

In [ ]:
# 将导入的数组转换为数据表并进行表头重命名
df = pd.DataFrame(curinfo)
df.rename(columns = 
    {
    0:'帐号'
    1:'卡号',     
    2:'平均油量升数', 
    3:'平均实际金额', 
    4:'平均折扣金额', 
    5:'平均余额', 
    6:'类型名称', 
    7:'交易类型描述', 
    8:'商品类型', 
    9:'商品种类名称', 
    10:'单位名称', 
    11:'首次交易日期', 
    12:'最近交易日期',
    13:'是否睡眠卡',
    14:'IC卡大额非油消费次数',
    15:'车队卡非油累计消费金额是否过高', 
    16:'车队卡非油累计消费次数是否过多', 
    17:'车队卡累计消费次数', 
    18:'车队卡累计消费金额', 
    19:'单日高频充值次数', 
    20:'车队卡汽油消费次数占比', 
    21:'车队卡柴油消费次数占比',
    22:'车队卡天然气消费次数占比', 
    23:'单日高频消费次数', 
    24:'车队卡汽油累计消费次数', 
    25:'车队卡汽油累计消费金额', 
    26:'单日夜间高频消费次数', 
    27:'加油卡非油累计消费金额', 
    28:'加油卡非油消费占总消费占比',
    29:'车队卡柴油是否高额消费'
    }, 
        inplace=True)

### 特征缩放

In [ ]:
# 将取值范围为0到1的字段*100
df_copy1 = df
df_copy1 = df_copy1.drop(['帐号', '首次交易日期', '最近交易日期'], axis=1)
for i in ['类型名称', '交易类型描述', '商品类型', '商品种类名称', '单位名称', '车队卡非油累计消费金额是否过高', 
          '车队卡非油累计消费次数是否过多','加油卡非油消费占总消费占比', '车队卡柴油是否高额消费', '是否睡眠卡']:
    df_copy1[i] = df_copy1[i] * 100

In [ ]:
# 将量级较大的字段归一化至范围为0到100
df_copy2 = df
min_max_scaler = preprocessing.MinMaxScaler(feature_range=(0,100))
x_minmax = min_max_scaler.fit_transform(df_copy2[['平均油量升数', '平均实际金额', '平均折扣金额', '平均余额', '车队卡累计消费金额',
                                                  '车队卡汽油累计消费金额','加油卡非油累计消费金额','IC卡大额非油消费次数',
                                                  '车队卡累计消费次数', '单日高频充值次数','单日高频消费次数', 
                                                  '车队卡汽油累计消费次数','单日夜间高频消费次数']])
df_minmax = pd.DataFrame(x_minmax)
df_minmax = df_minmax.rename(columns={0:'平均油量升数', 1:'平均实际金额', 2:'平均折扣金额', 3:'平均余额', 4:'车队卡累计消费金额', 
                                      5:'车队卡汽油累计消费金额', 6:'加油卡非油累计消费金额',7:'IC卡大额非油消费次数', 
                                      8:'车队卡累计消费次数', 9:'单日高频充值次数', 10:'单日高频消费次数', 
                                      11:'车队卡汽油累计消费次数',12:'单日夜间高频消费次数'})
df_minmax_concat1 = pd.concat([df_copy2.reset_index()['卡号'], df_minmax], axis = 1)
df_minmax_concat2 = pd.merge(df_minmax_concat1, df_copy1.drop(['平均油量升数', '平均实际金额', '平均折扣金额', '平均余额', '车队卡累计消费金额',
                                                  '车队卡汽油累计消费金额','加油卡非油累计消费金额','IC卡大额非油消费次数',
                                                  '车队卡累计消费次数', '单日高频充值次数','单日高频消费次数', 
                                                  '车队卡汽油累计消费次数','单日夜间高频消费次数'], axis=1), on='卡号')
df_minmax_concat2 = df_minmax_concat2.set_index('卡号')

### 主成分分析技术（PCA）

In [ ]:
# 创建一个PCA模型并适配数据表
pca = PCA(n_components = 5)
X2D = pca.fit_transform(df_minmax_concat2)

### 孤立森林算法

In [ ]:
# 读取训练好的模型
Iso_model = joblib.load('Iso_model.model')

# 预测
all_pred = Iso_model.predict(X2D[[0,1,2,3,4]])

X2D['anomaly'] = all_pred
# X2D.to_csv('outliers.csv', columns=["pred",], header=False)

outliers = X2D.loc[X2D['anomaly'] == -1]

In [ ]:
# 异常数据点的原数据表映射
df_outliers = df.iloc[list(outliers.index)]

In [ ]:
# 为卡号添加\t后缀，使得cvs可显示卡号全部位数
df_outliers['卡号'] = df_outliers['卡号'] + '\t'
# 将数据表导出
df_outliers.to_csv('车队卡异常加油卡.csv',encoding='utf-8-sig',index=False)